## Exploring the CCXT Library 

This notebook will explore the ccxt library documentation. We'll document how to use their public methods. 

In [ ]:
import ccxt
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from datetime import datetime
from tabulate import tabulate
import mplfinance as mpf

print(ccxt.exchanges)

In [ ]:
exchange = ccxt.binance()
# exchange.set_sandbox_mode(True) # enable sandbox mode

print(exchange.fetchTime()) # returns the current integer timestamp in milliseconds from the exchange server.
print(exchange.fetchStatus())

### Grabbing some of metadata and available functions from the exchange

In [ ]:
def print_exchange_info(exchange_id):
    # Create an instance of the exchange
    exchange_class = getattr(ccxt, exchange_id)
    exchange = exchange_class()

    # Print the exchange information
    print(f"Name: {exchange.name}")
    print(f"ID: {exchange.id}")
    print(f"Countries: {exchange.countries}")
    if exchange.has:
        for function_name, is_supported in exchange.has.items():
            if is_supported:
                print(function_name)

# Example usage:
print_exchange_info('binance')

#### Unified API is a subset of methods common among exchanges

### Price Tickers

A ticker is a statistical calculation with the information calculated over the past 24 hours for a specific market.

All fields in the ticker represent the past 24 hours prior to timestamp.

- The `bidVolume` is the volume (amount) of current best bid in the orderbook.
- The `askVolume` is the volume (amount) of current best ask in the orderbook.
- The `baseVolume` is the amount of base currency traded (bought or sold) in the last 24 hours.
- The `quoteVolume` is the amount of quote currency traded (bought or sold) in the last 24 hours.

In [ ]:
eth = exchange.fetchTicker('ETHUSDT')
eth

# Multiple Tickers
# multiple_tickers = exchange.fetchTickers(['BTCUSDT', 'LTCUSDT'])

# All the exchanges tickers at once. 
# all_tickers = exchange.fetchTickers()

### Fetching OHLCV

**Default number of bars returned is 500**

To obtain historical Mark, Index Price, and Premium Index candlesticks, pass the 'price' params-override to `fetchOHLCV`. The 'price' parameter accepts one of the following values:

- **'mark'**: It is the weighted average value of the underlying asset listed on major spot exchanges, which reflects the fair market value of the futures contract and is constantly updated to account for any changes in the asset’s spot price or the weighting of the exchanges used to calculate the index.
- **'index'**: The Price Index is the primary component of Mark Price. It is the weighted average value of the underlying asset listed on major spot exchanges, which reflects the fair market value of the futures contract and is constantly updated to account for any changes in the asset's spot price or the weighting of the exchanges used to calculate the index.
- **'premiumIndex'**:

In [ ]:
timeframe = '1d'  # 1 day
limit = 100  # Last 10 bars
ohlcv = exchange.fetch_ohlcv('ONDOUSDT', timeframe, limit=limit)

# Convert the data to a Pandas DataFrame and set the column names
ohlcv_df = pd.DataFrame(ohlcv, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])

# Convert the timestamp to a readable date format
ohlcv_df['Timestamp'] = pd.to_datetime(ohlcv_df['Timestamp'], unit='ms')

# Print the DataFrame in a tabulated format
print(ohlcv_df)

In [ ]:
mpf_style = 'nightclouds'
mpf.plot(ohlcv_df.set_index('Timestamp'), type='candle', style=mpf_style, volume=True, title='Candlestick')

#### Fetching Full price history

In [ ]:
def fetch_full_ohlcv_history(exchange, symbol, timeframe):
    all_ohlcv = []
    since = 0  # Set to 0 to start from the earliest available data
    while True:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since)
        if not ohlcv:
            break
        all_ohlcv.extend(ohlcv)
        since = ohlcv[-1][0] + 1  # Increment since to the timestamp of the last fetched bar
    return all_ohlcv

# Fetch the full OHLCV history
full_ohlcv = fetch_full_ohlcv_history(exchange, 'OPUSDT', timeframe)

# Convert the full OHLCV history into a DataFrame
ohlcv_df = pd.DataFrame(full_ohlcv, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])

# Convert the 'Timestamp' column to a datetime format
ohlcv_df['Timestamp'] = pd.to_datetime(ohlcv_df['Timestamp'], unit='ms')

# Print
print(ohlcv_df)

### Orderbook

Exchanges expose information on open orders with bid (buy) and ask (sell) prices, volumes and other data. Usually there is a separate endpoint for querying current state (stack frame) of the order book for a particular market. An order book is also often called market depth. The order book information is used in the trading decision making process.

1. `fetchOrderBook(symbol, limit=None, params={})` to fetch a single market's order book.
2. `fetchOrderBooks(symbols)` to fetch order books for multiple markets.
3. `fetchOrderBooks()` to fetch order books for all markets if the exchange supports this.

In [ ]:
symbol = 'BTC/USDT'  # Replace with the symbol you want to query
limit = 3  # Replace with the limit you want, or keep as None for default
params = {}  # Replace with any additional parameters if needed
order_book_single = exchange.fetchOrderBook(symbol, limit, params)

print("Single Market Order Book:")
print(json.dumps(order_book_single, indent=4))

### Ploting the market depth chart

In [ ]:
order_book = exchange.fetch_order_book('BTC/USDT', limit=100)
bids = order_book['bids']
asks = order_book['asks']

bids = sorted([(float(price), float(quantity)) for price, quantity in bids], reverse=True)
asks = sorted([(float(price), float(quantity)) for price, quantity in asks])

bid_prices, bid_cumulative = zip(*[(price, sum(quantity for p, quantity in bids if p >= price)) for price, _ in bids])
ask_prices, ask_cumulative = zip(*[(price, sum(quantity for p, quantity in asks if p <= price)) for price, _ in asks])

mid_price = (bid_prices[0] + ask_prices[0]) / 2

plt.style.use('dark_background')
plt.figure(figsize=(10, 6))
plt.plot(bid_prices, bid_cumulative, label='Bids', color='green')
plt.plot(ask_prices, ask_cumulative, label='Asks', color='red')
plt.fill_between(bid_prices, bid_cumulative, color='green', alpha=0.3)
plt.fill_between(ask_prices, ask_cumulative, color='red', alpha=0.3)
plt.title('BTCUSDT Market Depth')
plt.xlabel('Price')
plt.ylabel('Cumulative Quantity')
plt.legend()
plt.annotate(f'Mid Price: {mid_price}', xy=(mid_price, 0), xytext=(mid_price, max(max(bid_cumulative), max(ask_cumulative))/2),
             arrowprops=dict(facecolor='yellow', shrink=0.05), horizontalalignment='center')
plt.show()

### Public Trades

Public trades represent the most recent transactions that have occurred on an exchange for a particular trading pair, such as BTC/USD. These trades are public because they have been executed and are now part of the market's transaction history. Traders and analysts often use this data to understand the current market sentiment, to analyze the trading volume, and to detect patterns or trends in the price movement.

In [ ]:
symbol = 'BTC/USD'  # Replace with the desired trading pair
limit = 20

public_trades = exchange.fetch_trades(symbol, limit=limit)

# Prepare the data for tabulation
headers = ["Trade ID", "Timestamp", "Symbol", "Type", "Side", "Taker or Maker", "Price", "Amount"]
trade_data = [
    [
        trade['id'],
        f"{trade['timestamp']} ({trade['datetime']})",
        trade['symbol'],
        trade['type'],
        trade['side'],
        trade['takerOrMaker'],
        trade['price'],
        trade['amount']
    ]
    for trade in public_trades
]

# Print the trades in a tabulated format
print(tabulate(trade_data, headers=headers))

### Fetching the Funding Rate

Data on the current, most recent, and next funding rates can be obtained using the methods

- fetchFundingRates () for all market symbols
- fetchFundingRates ([ symbol1, symbol2, ... ]) for multiple market symbols
- fetchFundingRate (symbol) for a single market symbol


In [ ]:
if exchange.has['fetchFundingRates']:
    # Fetch funding rates for all market symbols
    all_funding_rates = exchange.fetchFundingRates()
    # print("All funding rates:", all_funding_rates)
else:
    print(f"The exchange {exchange_id} does not support fetchFundingRates.")

# To fetch funding rates for multiple specific symbols
symbols = ['BTC/USDT:USDT', 'ETH/USDT:USDT']  # Replace with your symbols of interest
if exchange.has['fetchFundingRates']:
    multiple_funding_rates = exchange.fetchFundingRates(symbols)
    # print(f"Funding rates for {symbols}:", multiple_funding_rates)
else:
    print(f"The exchange {exchange_id} does not support fetchFundingRates for multiple symbols.")

# To fetch the funding rate for a single market symbol
symbol = 'BTC/USDT:USDT'  # Replace with your symbol of interest
if exchange.has['fetchFundingRate']:
    single_funding_rate = exchange.fetchFundingRate(symbol)
    # Format the single funding rate in nicely formatted JSON
    formatted_single_funding_rate = json.dumps(single_funding_rate, indent=4)
    print(f"Funding rate for {symbol} in JSON format:\n{formatted_single_funding_rate}")
else:
    print(f"The exchange {exchange_id} does not support fetchFundingRate for a single symbol.")

### Funding Rate History Plotted

In [ ]:
symbol = 'BTC/USDT:USDT'  # Replace with the actual symbol you want to fetch
since = None  # Replace with the actual timestamp or leave as None
limit = 100  # Replace with the desired limit or leave as None
params = {}  # Replace with any additional params or leave as empty dict

# Fetch the funding rate history
funding_rate_history = exchange.fetchFundingRateHistory(symbol, since, limit, params)

# Extract timestamps and funding rates
timestamps = [entry['timestamp'] for entry in funding_rate_history]
funding_rates = [entry['fundingRate'] for entry in funding_rate_history]

# Convert timestamps to matplotlib dates
dates = [mdates.date2num(datetime.utcfromtimestamp(timestamp / 1000)) for timestamp in timestamps]

# Plot the funding rates
plt.figure(figsize=(10, 5))
plt.plot(dates, funding_rates, label='Funding Rate')  # Plot the funding rates over time

# Format the x-axis to show fewer date labels and format them properly
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gcf().autofmt_xdate()  # Auto-format the x-axis labels for better readability

# Include a label of the last funding rate as a percentage and the date time
if funding_rates:
    last_funding_rate = funding_rates[-1]
    last_funding_rate_percentage = last_funding_rate * 100
    last_date = datetime.utcfromtimestamp(timestamps[-1] / 1000).strftime('%Y-%m-%d %H:%M:%S')
    plt.annotate(f'{last_funding_rate_percentage:.4f}%\n{last_date}',  # Format to 4 decimal places and include date time
                 xy=(dates[-1], last_funding_rate),
                 xytext=(15, 0),  # Shift the text slightly to the right
                 textcoords='offset points',
                 ha='center', va='bottom')

plt.title('Funding Rate History')
plt.xlabel('Date')  # Label for the x-axis
plt.ylabel('Funding Rate')  # Label for the y-axis
plt.legend()  # Show the legend
plt.tight_layout()  # Adjust layout to prevent clipping

### Open Interest

Open interest is a measure of the total number of outstanding derivative contracts, such as futures or options, that have not been settled. For traders, open interest provides useful information that can help them understand the market sentiment. An increasing open interest indicates that new money is flowing into the market, suggesting a strong trend and that the current trend is likely to continue. Conversely, decreasing open interest implies that the market is liquidating and may suggest that the current trend is nearing its end.

Traders can use open interest in combination with other indicators to confirm trends and reversals. For example, if the price is rising and open interest is also increasing, it could be interpreted as a bullish sign. If the price is falling and open interest is increasing, it could be seen as a bearish sign. However, if the price is rising and open interest is decreasing, traders might take it as a sign that the trend is not supported by new buyers and could reverse soon.


In [ ]:
symbol = 'BTC/USDT'  # Replace with the actual symbol you want to query

# Fetch open interest for the specified symbol
open_interest = exchange.fetchOpenInterest(symbol)

# Print the open interest in formatted JSON
print(json.dumps({"Open Interest for": symbol, "Value": open_interest}, indent=4))

### Open Interest History

Parameters

- symbol (String) Unified CCXT market symbol (e.g. "BTC/USDT:USDT")
- timeframe (String) Check exchange.timeframes for available values
- since (Integer) Timestamp for the earliest open interest record (e.g. 1645807945000)
- limit (Integer) The maximum number of open interest structures to retrieve (e.g. 10)
- params (Dictionary) Extra parameters specific to the exchange API endpoint (e.g. {"endTime": 1645807945000})

Note for OKX users: instead of a unified symbol okx.fetchOpenInterestHistory expects a unified currency code in the symbol argument (e.g. 'BTC').
The format of the symbol should be 'BTC/USDT:USDT'

In [ ]:
symbol = 'BTC/USDT:USDT'

# Timeframe for the open interest data
timeframe = '1d'

try:
    # Fetch the open interest history
    open_interest_history = exchange.fetch_open_interest_history(symbol, timeframe)
except Exception as e:
    print(f"An error occurred: {e}")
    open_interest_history = []  # Set to empty list in case of an error

# Proceed only if open_interest_history is not empty
if open_interest_history:
    # Extract timestamps and open interest values
    timestamps = [data['timestamp'] for data in open_interest_history]
    # Corrected the key error by using the correct key for open interest values
    # The correct key is 'openInterestValue' as per the command
    open_interest_values = [float(data['info']['sumOpenInterestValue']) for data in open_interest_history]

    # Convert timestamps to readable dates using the exchange's iso8601 method
    dates = [exchange.iso8601(timestamp) for timestamp in timestamps]

    plt.figure(figsize=(12, 6))
    plt.plot(dates, open_interest_values, label='Open Interest')
    plt.title('Open Interest History')
    plt.xlabel('Date')
    plt.ylabel('Open Interest Value')
    plt.legend()
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    plt.show()